<img src="Imagenes/Mac_wallpaper_3.png" width="50%">

In [ ]:
!pip install "qiskit[visualization]" --user

In [ ]:
!pip install qiskit-aer --user

# Algoritmo de Grover

Para finalizar con este taller introductorio, vamos a ver un algoritmo que permite acelerar el proceso de búsqueda en una base de datos sin estructura, aunque sus usos se extienden más allá de eso. Por ejemplo, puede ser utilizado como una subrutina para obtener una ventaja de velocidad similar en otros algoritmos.

### Introducción

Supongamos que tenemos una larga lista de N elementos. Entre estos elementos hay uno con una propiedad única que queremos localizar, lo llamaremos el "ganador" $w$. Piensa en cada elemento de la lista como una caja de un color en particular. Digamos que todos los elementos en la lista son grises excepto el ganador $w$, que es morado (imagen extraída del textbook de Qiskit): 

<img src="Imagenes/Lista.png" width="50%">

Para encontrar la caja morada (el elemento marcado) utilizando una computadora clásica, tendríamos que revisar (en promedio) $N/2$ de estas cajas, y en el peor de los casos, todas las $N$ cajas.

El algoritmo de Grover nos permite encontrar el elemento marcado en aproximadamente $\sqrt{N}$ pasos, una ventaja cuadrática respecto del caso clásico. Y como una ventaja adicional, el algoritmo no utiliza la estructura interna de la lista para realizar la búsqueda, lo que lo hace **genérico**, lo que lo convierte en una herramienta bastante útil.

### Pasos del algoritmo

El algoritmo de Grover consiste de tres pasos principales (imagen extraída del textbook de Qiskit):<br>
- Preparación de los estados.
- El oráculo.
- El operador de difusión.

<img src="Imagenes/Grover.png" width="50%">

La preparación de los estados es el paso en el que creamos el espacio de búsqueda, que está compuesto por todos los posibles valores que puede tomar el elemento que estamos buscando. En el ejemplo de una lista , el espacio de búsqueda serían todos los elementos de dicha lista. 

El oráculo es el que marca la respuesta (o respuestas) correcta(s).

Y el operador de difusión amplifica la amplitud de estas respuestas para aumentar la probabilidad de obtenerlas como el resultado de la medición al final del algoritmo. Esta es la parte central del algoritmo de Grover, y es conocido como el truco de amplificación de amplitud.

Los pasos de aplicación del oráculo y el operador de difusión se deben repetir cierta cantidad de veces $k \propto \sqrt{N}$ para obtener el resultado deseado al final del algoritmo. 

### Aplicación del algoritmo

**Primer paso:** Para crear el espacio de búsqueda solo necesitamos utilizar una superposición equitativa (para que inicialmente todos los elementos tengan la misma amplitud), que podemos construir fácilmente utilizando compuertas Hadamard sobre todos nuestros qubits. Llamaremos a este estado inicial $|s\rangle$, por lo que  

$\hspace{9 cm} |s\rangle = H^{\otimes n}|0^{\otimes n}\rangle = \dfrac{1}{\sqrt{2^{n}}}\displaystyle\sum_{x\in\{0,1\}^{n}}|x\rangle$

*Nota: Podemos empezar en cualquier otro estado que posea una superposición equitativa, dependiendo de la información que tengamos sobre la respuesta que estamos buscando. Nosotros veremos el caso más general ya que no realizaremos ejemplos demasiado complicados, pero en la práctica es común reducir el espacio de búsqueda si es posible.*

Además, definimos dos estados perpendiculares entre sí:

$\hspace{11 cm} |A\rangle = \dfrac{1}{\sqrt{a}}\displaystyle\sum_{x\in A}|x\rangle$

$\hspace{11 cm} |B\rangle = \dfrac{1}{\sqrt{b}}\displaystyle\sum_{x\in B}|x\rangle$

Donde $A$ es el conjunto de las respuestas correctas, que contiene $a$ elementos; y $B$ es el conjunto de los demás elementos de $|s\rangle$ que no están en $A$, y contiene $b$ elementos. De esta forma, si tenemos $N = 2^{n}$ elementos en nuestro espacio de búsqueda, entonces

$\hspace{10 cm} |s\rangle = \sqrt{\dfrac{a}{N}}|A\rangle + \sqrt{\dfrac{b}{N}}|B\rangle$

---

Ahora, para entender por completo cuál es el efecto de los siguientes dos pasos sobre nuestro sistema, vamos a cambiar nuestro marco de referencia. Ya que nuestros estados $|A\rangle$ y $|B\rangle$ son ortogonales, estos generan un plano en dos dimensiones, el cual vamos a utilizar como nuestro marco de referencia. Ya que $|s\rangle$ es una combinación lineal de $|A\rangle$ y $|B\rangle$ este se encuentra en el plano que generan estos dos estados. (imagen extraída del textbook de Qiskit, en el cual $w=A$ y $s^{\prime} = B$)

<img src="Imagenes/Geom.png" width="30%">

Por lo que en este nuevo marco de referencia podemos escribir a nuestro estado $|s\rangle$ como

$\hspace{10 cm} |s\rangle = \sin\theta|A\rangle + \cos\theta|B\rangle$

donde

$\hspace{11 cm} \theta = \arcsin\left(\sqrt{\dfrac{a}{N}}\right)$

---

**Segundo paso:** Aplicamos el oráculo. Ya que queremos "marcar" a nuestras respuestas correctas dentro de la superposición equitativa, definiremos un operador $Z_{f}$ que actuará de la siguiente forma:

$\hspace{9 cm} Z_{f}|A\rangle = -|A\rangle \hspace{2 cm} Z_{f}|B\rangle = |B\rangle$

De esta manera, utilizamos un cambio de fase relativa para distinguir a nuestras respuestas del resto de elementos de la superposición.

**Tercer paso:** Aplicamos el operador de difusión, el cual tiene la forma

$\hspace{11 cm} D = 2|s\rangle\langle s|- \mathbb{1}$

Primero vamos a evaluar su acción sobre nuestro estado $|A\rangle$:

$\hspace{5 cm} D|A\rangle = 2|s\rangle\langle s|A\rangle - |A\rangle = 2\sin\theta|s\rangle - |A\rangle = \left(2\sin^{2}\theta - 1\right)|A\rangle + 2\sin\theta\cos\theta|B\rangle$

Ahora, veamos cómo actúa sobre nuestro estado $|B\rangle$:

$\hspace{5 cm} D|B\rangle = 2|s\rangle\langle s|B\rangle - |B\rangle = 2\cos\theta|s\rangle - |B\rangle = \left(2\cos^{2}\theta - 1\right)|B\rangle + 2\sin\theta\cos\theta|A\rangle$

---

Ya que los pasos del oráculo y del operador de difusión se deben repetir varias veces, vamos a definir un operador $G$ que los junte a ambos, es decir:

$\hspace{10 cm} G = DZ_{f} = (2|s\rangle\langle s|- \mathbb{1})Z_{f}$

Entonces, el efecto de $G$ sobre nuestros estados $|A\rangle$ y $|B\rangle$ es

$\hspace{3 cm} G|A\rangle = (1 - 2\sin^{2}\theta)|A\rangle - 2\sin\theta\cos\theta|B\rangle = (\cos^{2}\theta - \sin^{2}\theta)|A\rangle - \sin 2\theta|B\rangle = \cos 2\theta|A\rangle - \sin 2\theta|B\rangle$

$\hspace{6 cm} G|B\rangle = (\cos^{2}\theta - \sin^{2}\theta)|B\rangle + 2\sin\theta\cos\theta|A\rangle = \cos 2\theta|B\rangle + \sin 2\theta|A\rangle$

Por lo que el efecto del operador $G$ sobre el estado $|s\rangle$ (que recordemos es el estado de nuestro sistema completo) es

$G|s\rangle = \sin\theta(\cos 2\theta|A\rangle - \sin 2\theta|B\rangle) + \cos\theta(\cos 2\theta|B\rangle + \sin 2\theta|A\rangle) = (\sin\theta\cos 2\theta + \cos\theta\sin 2\theta)|A\rangle + (\cos\theta\cos 2\theta - \sin\theta\sin 2\theta)|B\rangle = \hspace{2 cm} \sin 3\theta|A\rangle + \cos 3\theta |B\rangle$

¡El efecto del operador $G$ es hacer una rotación de $2\theta$ radianes en el plano! Sabiendo esto, podemos deducir cuál será el efecto de aplicar dicho operador $k$ veces:

$\hspace{8 cm} G^{k}|s\rangle = \sin[(2k + 1)\theta]|A\rangle + \cos[(2k + 1)\theta]|B\rangle$

Ya que queremos que al final del algoritmo el estado $|A\rangle$ tenga una amplitud lo suficientemente grande comparada con la amplitud de $|B\rangle$ para asegurarnos de que lo vamos a medir, necesitamos que

$\hspace{11 cm} \sin[(2k + 1)\theta] \approx 1$

por lo que

$\hspace{9 cm} (2k + 1)\theta \approx \dfrac{\pi}{2} \hspace{2 cm} k \approx \dfrac{\pi}{4\theta} - \dfrac{1}{2}$

Ahora solo falta el valor de $\theta$ para saber cuántas veces debemos aplicar estos dos pasos del algoritmo de Grover para obtener la respuesta que buscamos. Para ello, vamos a asumir que $\sqrt{\dfrac{a}{N}} \ll 1$ de manera que $\theta \approx \sqrt{\dfrac{a}{N}}$, y entonces

$\hspace{10 cm} k \approx \dfrac{\pi}{4}\sqrt{\dfrac{N}{a}} - \dfrac{1}{2} \approx \dfrac{\pi}{4}\sqrt{\dfrac{N}{a}}$

Y en el caso de que solo tengamos una respuesta correcta, es decir, $a=1$:

$\hspace{11 cm} k \approx \dfrac{\pi\sqrt{N}}{4}$

### Aplicar el operador de difusión

El último paso antes de aplicar el algoritmo de Grover es encontrar una manera de escribir el operador de difusión $D = 2|s\rangle\langle s|- \mathbb{1}$ como una combinación de compuertas cuánticas que podamos aplicar en nuestro circuito. Para ello, vamos a recordar que $|s\rangle = H^{\otimes n}|0^{\otimes n}\rangle$ y que las compuertas Hadamard son su propio inverso para reescribir el operador de difusión como 

$\hspace{6 cm} D = 2H^{\otimes n}|0^{\otimes n}\rangle\langle 0^{\otimes n}|H^{\otimes n}- H^{\otimes n}H^{\otimes n} = H^{\otimes n}(2|0^{\otimes n}\rangle\langle 0^{\otimes n}| - \mathbb{1})H^{\otimes n}$

Ahora, si investigamos la forma matricial del operador $2|0^{\otimes n}\rangle\langle 0^{\otimes n}| - \mathbb{1}$ nos daremos cuenta de que es una matriz identidad con todas las entradas de su diagonal multiplicadas por $-1$ excepto la primera (la correspondiente al estado de solo ceros). Esto significa que su efecto es multiplicar todos los estados por $-1$ excepto el estado de solo ceros, el cual deja intacto.  

Ya que esto sería difícil de implementar en un circuito, vamos a utilizar el operador $-D$ en vez del operador $D$. Esto lo podemos hacer ya que la diferencia entre ambos es una fase global, y las fases globales no afectan en nada a nuestros estados más allá de cambiar la forma de su vector de estado. Además, al aplicar el operador $-D$ estaríamos aplicando el operador $\mathbb{1} - 2|0^{\otimes n}\rangle\langle 0^{\otimes n}|$, el cual es una matriz identidad cuya primera entrada en la diagonal es un $-1$ en vez de un $1$; es decir, su efecto es multiplicar el estado de solo ceros por $-1$ y dejar a los demás estados intactos. El comportamiento es el mismo, marcar el estado de solo ceros, solo que con el operador $-D$ esta tarea es mucho más sencilla y se puede conseguir con una combinación de compuertas $X$ y una compuerta $Z$ multicontrolada, de manera que

$\hspace{7 cm} -D = H^{\otimes n}(\mathbb{1} - 2|0^{\otimes n}\rangle\langle 0^{\otimes n}|)H^{\otimes n} = H^{\otimes n}(X^{\otimes n}CZX^{\otimes n})H^{\otimes n}$

### El circuito cuántico

Ahora sí, vamos a poner a prueba el algoritmo de Grover con dos casos sencillos. En el primero, vamos a buscar el elemento $1101$. En este caso, $a=1$ y $N=16$, por lo que $k\approx \pi$. En este caso, tomaremos $k=3$ y veremos qué resultado obtenemos.

El primer paso es crear nuestro oráculo. Ya que queremos marcar el estado $1101$ solo debemos aplicar una compuerta $X$ al qubit en la posición 1 y luego aplicar una compuerta $Z$ multicontrolada. Para implementarla vamos a utilizar una compuerta Toffoli multicontrolada y compuertas Hadamard sobre nuestro qubit objetivo. La sintaxis para una compuerta Toffoli multicontrolada es `mcx(controles,objetivo)`.

In [ ]:
from qiskit import QuantumCircuit, execute, Aer

def oracle(qc):
    qc.x(1)
    qc.h(0)
    qc.mcx([3,2,1],0)
    qc.h(0)
    qc.x(1)

Ahora definimos nuestro operador de difusión.

In [ ]:
def diffuser(qc,n):
#Primer paso
    for i in range(n):
        qc.h(i)
        qc.x(i)
#Segundo paso
    qc.h(n-1)
    qc.mcx(list(range(n-1)),n-1)
    qc.h(n-1)
#Tercer paso
    for i in range(n):
        qc.x(i)
        qc.h(i)

Finalmente, lo juntamos todo y recordamos que debemos aplicar el oráculo y el operador de difusión **3** veces.

In [ ]:
n = 4

qc = QuantumCircuit(n,n)

# Inicializamos nuestros estados en una superposición equitativa

for i in range(n):
    qc.h(i)
    
# Aplicamos el oráculo y el difusor tres veces

for i in range(3):
    oracle(qc)
    diffuser(qc,n)
    
# Medimos nuestro circuito

qc.measure([0,1,2,3],[0,1,2,3])
    
# Ejecutamos nuestro circuito y obtenemos los resultados

job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100)
counts = job.result().get_counts(qc)
print(counts)

¡Listo! Y ahora, como segundo ejemplo, marquemos las cadenas de bits $110$ y $111$. En este caso, nuestro oráculo es más sencillo, ya que solo debemos implementar una compuerta $Z$ controlada para marcar ambos estados.

In [ ]:
def oracle(qc):
    qc.cz(1,2)

Ahora, calculamos $k$ sabiendo que $a=2$ y $N=8$. Entonces $k \approx \pi/2$, por lo que de momento tomaremos $k=1$ y veremos si es necesario subir a $k=2$ o no.

In [ ]:
n = 3

qc = QuantumCircuit(n,n)

# Inicializamos nuestros estados en una superposición equitativa

for i in range(n):
    qc.h(i)
    
# Aplicamos el oráculo y el difusor una vez

oracle(qc)
diffuser(qc,n)
    
# Medimos nuestro circuito

qc.measure([0,1,2],[0,1,2])
    
# Ejecutamos nuestro circuito y obtenemos los resultados

job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100)
counts = job.result().get_counts(qc)
print(counts)